In [3]:
# importing libraries
from PIL import Image
from numpy.linalg import inv
from matplotlib import pyplot as plt
from matplotlib.image import imread
import numpy as np
import os

In [4]:
img_path = 'images_assgn/'

# cropping the images to 512x512 (Removing white space around images)
# acquired co-ordinates using MSpaint(Grid & Scaling)
coords = (82, 30, 594, 542)
for i in range(1,5):
    image_obj = Image.open(img_path+str(i)+'.jpg')
    cropped_image = image_obj.crop(coords)
    cropped_image.save(img_path+str(i)+'_cropped.jpg')

imageList = np.ndarray(shape=(512,512,4), dtype=np.integer)

# Loading the training images
for i in range(4):
    img = plt.imread(img_path + str(i+1)+'_cropped.jpg')
    imageList[:,:,i] = np.array(img[:,:,0])

In [5]:
# Taking random sample points using MSpaint(Grid & Scaling)
x_river = [159,158,153,154,163,167,171,217,160,213,173,163,185,216,159,162,186,170,150,159,
           186,211,164,218,228,156,179,151,173,145,185,186,181,213,159,208,
          158,191,224,218,178,170,183,171,218,214,218,151,158,225]
y_river = [6,38,16,7,30,129,166,292,32,287,127,159,254,286,156,156,233,461,12,45,229,347,47,324,322,16,120,
           6,457,19,233,232,228,282,18,292,20,247,314,344,120,34,233,128,351,287,280,7,9,313]
x_non_river = [32,52,348,485,231,123,343,234,435,372,325,141,263,248,72,237,283,492,132,86,116,224,298,479,238,259,431,461,411,277,388,206,469,392,129,371,
            108,426,111,424,415,426,101,124,398,45,476,78,268,335,372,124,304,58,357,338,270,440,310,281,428,222,468,417,332,321,469,443,410,410,441,68,
            235,460,463,375,68,454,314,316,274,101,114,362,390,257,324,292,474,455,468,455,473,230,413,360,55,313,418,488]
y_non_river = [436,252,476,307,93,116,490,468,400,452,307,309,35,301,450,148,41,450,295,452,312,211,36,430,56,51,446,490,29,480,99,55,408,91,126,175,
            401,483,142,410,189,302,268,112,121,432,221,260,348,412,159,406,220,446,307,476,144,416,96,246,24,185,90,235,252,440,52,116,388,104,62,128,
            161,480,422,287,130,318,143,357,93,440,279,303,58,224,347,74,35,452,56,458,463,60,306,334,145,470,40,184]

In [6]:
# calculating means
# River Class : T1 = [Mean1; Mean2; Mean3; Mean4];
T1 = np.zeros(shape=(4), dtype=np.float64)
for i in range(50):
    for j in range(4):
        T1[j] = T1[j] + imageList[x_river[i],y_river[i],j]
T1 = T1/50

# NonRiver Class : T2 = [Mean1; Mean2; Mean3; Mean4];
T2 = np.zeros(shape=(4), dtype=np.float64)
for i in range(100):
    for j in range(4):
        T2[j] = T2[j] + imageList[x_non_river[i],y_non_river[i],j]
T2 = T2/100

In [41]:
# calculating co-variance matrices
riverMat=np.ndarray(shape=(50,4))
non_riverMat=np.ndarray(shape=(100,4))

# Deviation matrices of river and non_river data
for i in range(50):
    riverMat[i] = np.subtract(imageList[x_river[i],y_river[i],:],T1)
for i in range(100):
    non_riverMat[i] = np.subtract(imageList[x_non_river[i],y_non_river[i],:],T2)
    
river_cov_mat = np.ndarray(shape=(4, 4), dtype=np.float64)
non_river_cov_mat = np.ndarray(shape=(4, 4), dtype=np.float64)

# covariance matrices using deviation matrices
for i in range(4):
    for j in range(4):
        river_cov_mat[i][j]=np.dot(riverMat[:,i],riverMat[:,j])/50

for i in range(4):
    for j in range(4):
        non_river_cov_mat[i][j]=np.dot(non_riverMat[:,i],non_riverMat[:,j])/100

In [42]:
# Applying Baye's decision rule
P = [0.3, 0.5, 0.7]
for x in range(3):
    P1=P[x]
    P2=1-P1
    output=np.zeros(shape=(512, 512), dtype = np.integer)
    for i in range(512):
        for j in range(512):
            dev_riv = np.subtract(imageList[i,j,:],T1)
            dev_non_riv = np.subtract(imageList[i,j,:],T2)
            river_class = np.dot(np.dot(dev_riv.T,inv(river_cov_mat)), dev_riv)
            non_river_class = np.dot(np.dot(dev_non_riv.T,inv(non_river_cov_mat)), dev_non_riv)
            p1 = (-0.5) * np.exp(river_class)/np.sqrt(np.linalg.det(river_cov_mat));             # calculating probability
            p2 = (-0.5) * np.exp(non_river_class)/np.sqrt(np.linalg.det(non_river_cov_mat));     # densities
            # Bayes's Decision
            if((P1 * p1) >= (P2 * p2)):
                output[i,j]=255
    # saving the output images in output folder
    plt.imsave('output/output'+str(x+1)+'.png',output,cmap='gray')